BASE RELATIONNELLE

In [1]:
import pandas as pd
import numpy as np
import sqlite3, sqlalchemy
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect
from IPython.display import Markdown, display
%load_ext sql
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False
%config SqlMagic.displaylimit = 50
%sql sqlite:///trustscore.db



In [2]:
#Infos d'entreprises

df_enterprises = pd.read_csv(r"C:\Users\MSI KATANA B13V\Desktop\PARCOURS DE\Projet Fil Rouge\de_satisfaction_client\data\informations_entreprises.csv", sep=',')
df_enterprises.head()

,review,town,country,institution_type,five_star_%,company_name,trust_score
0,327,Portland,United States,Financial Institution,96%,Evergreen Credit Union,4.9
1,290,Lincoln,United States,Financial Institution,92%,Liberty First Credit Union,4.7
2,3242,Irvine,United States,Financial Institution,66%,GoHenry by Acorns,4.3
3,175,Chicago,United States,Alternative Financial Service,60%,Crypto Dispensers,3.9
4,10,San Francisco,United States,Financial Institution,100%,Swadesh,4.4


In [29]:
#Infos commentaires

df_comments = pd.read_json(r"C:\Users\MSI KATANA B13V\Desktop\PARCOURS DE\Projet Fil Rouge\de_satisfaction_client\data\commentaires.json")
df_comments.head()

,company_name,User,localisation,Titre,commentaire,nombre_reviews,date_experience,reply
0,Evergreen Credit Union,NA,US,All my questions were answered in a…,All my questions were answered in a timely man...,1,"June 07, 2024",None
1,Evergreen Credit Union,NA,US,Best Ever Banking Experience,Casey has been amazing to work with. I have be...,1,"May 23, 2024","{'reply_text': 'Hi Lindsay, We're glad Casey ..."
2,Evergreen Credit Union,NA,US,Rebecca did a wonderful job,Rebecca did a wonderful job! My problem was ha...,1,"May 31, 2024",None
3,Evergreen Credit Union,NA,US,Exceptional Service From Entering to Exiting!,We wanted to extend our gratitude to the excep...,1,"May 10, 2024","{'reply_text': 'Thank you, Noyes Family! We're..."
4,Evergreen Credit Union,NA,US,I was anxious when I called egcu.org on…,I was anxious when I called egcu.org on Monday...,1,"May 01, 2024","{'reply_text': 'Hi Linda, Thank you very much..."


In [3]:
# Connexion à la base de données SQLite
conn = sqlite3.connect('trustscore.db')
cursor = conn.cursor()

# Création de la table `enterprise`
cursor.execute('''
CREATE TABLE IF NOT EXISTS companies (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    company_name TEXT,
    town TEXT,
    country TEXT,
    institution_type TEXT,
    review REAL,
    trust_score REAL,
    five_star_percentage REAL
)
''')

# Fermer la connexion
conn.commit()
conn.close()


NETTOAYGE ET INSERTION DES DONNEES

In [4]:
import pandas as pd
import sqlite3

df_enterprises = pd.DataFrame(df_enterprises)

# Nettoyage des données
df_enterprises['five_star_percentage'] = df_enterprises['five_star_%'].str.rstrip('%').astype(float)

# Supprimer l'ancienne colonne 'five_star_%'
df_enterprises.drop(columns=['five_star_%'], inplace=True)

# Connexion à la base de données SQLite
conn = sqlite3.connect('trustscore.db')

# Insérer les données dans la table `enterprise`
df_enterprises.to_sql('companies', conn, if_exists='append', index=False)

# Fermer la connexion
conn.close()


In [7]:
%%sql 

select * from companies limit 5; 

id,company_name,town,country,institution_type,review,trust_score,five_star_percentage
1,Evergreen Credit Union,Portland,United States,Financial Institution,324.0,4.9,96.0
2,Liberty First Credit Union,Lincoln,United States,Financial Institution,290.0,4.7,92.0
3,GoHenry by Acorns,Irvine,United States,Non-Bank Financial Service,3239.0,4.3,66.0
4,Crypto Dispensers,Chicago,United States,Cryptocurrency Service,176.0,3.9,60.0
5,Swadesh,San Francisco,United States,Financial Institution,10.0,4.4,100.0


In [6]:
%%sql

select distinct company_name

from companies; 



company_name
Evergreen Credit Union
Liberty First Credit Union
GoHenry by Acorns
Crypto Dispensers
Swadesh
The Payment HQ
Meriwest Credit Union
ASDFTEST
ATM Together
WPC Merchant Services & Credit Card Processing


In [9]:
%%sql

select avg(review) as _avg_review,
       town,
       company_name
from companies
GROUP BY town,company_name; 

_avg_review,town,company_name
1.0,None,ASDFTEST
0.0,None,Covault
12.0,None,Heritage Valley Federal Credit Union
3.0,None,cryptobaseatm.com
0.0,Bronx,Slide2Thrive
0.0,Carlsbad,ATMDepot.com
175.5,Chicago,Crypto Dispensers
0.0,Chicago,Liberty Bank for Savings
19.0,Cookeville,Cash Express
0.0,Fort Lauderdale,Nationwide Payment Systems


BASE ORIENTEE DOCUMENT

Etape 1 : "Il faut s'assurer que MongoDB est installé et actif."
Etape 2 : " Lancer la commande docker-compose up -d  qui va initialiser le Docker avec tous les composants nécéssaire voir fichier .yml"

In [11]:
import json
from pymongo import MongoClient

# Chargement du fichier sur les commentaires
json_file = r'C:\Users\MSI KATANA B13V\Desktop\PARCOURS DE\Projet Fil Rouge\de_satisfaction_client\data\commentaires.json'

# Connexion à MongoDB (vérifier que Docker est en cours d'exécution)
client = MongoClient('mongodb://localhost:27017/')

# Sélection de la base de données
db = client['trustscore']

# Sélection de la collection (ou création si elle n'existe pas)
collection = db['commentaires']

# Lecture du fichier JSON et insertion des données dans la collection
with open(json_file, 'r', encoding='utf-8') as file:
    data = json.load(file)
    collection.insert_many(data)

# Fermeture de la connexion à MongoDB
client.close()

print("Insertion des données terminée.")


Insertion des données terminée.


In [22]:
import json
from pymongo import MongoClient

# Connexion à MongoDB (vérifier que Docker est en cours d'exécution)
client = MongoClient('mongodb://localhost:27017/')

#print(client.list_database_names())

comments = client["trustscore"]["commentaires"]

for i in list(comments.find().distinct("localisation")):
    print(i)



for i in list(comments.find({},{"company_name":"Evergreen Credit Union","localisation":"US"}).limit(20)):
    print(i)



# Fermeture de la connexion à MongoDB
client.close()

AU
BD
ET
GB
GR
ID
IN
KH
MM
MX
NG
PK
SE
TH
TR
UA
US
ZA
{'_id': ObjectId('66718e3343c23fc600fc5ec8'), 'company_name': 'Evergreen Credit Union', 'localisation': 'US'}
{'_id': ObjectId('66718e3343c23fc600fc5ec9'), 'company_name': 'Evergreen Credit Union', 'localisation': 'US'}
{'_id': ObjectId('66718e3343c23fc600fc5eca'), 'company_name': 'Evergreen Credit Union', 'localisation': 'US'}
{'_id': ObjectId('66718e3343c23fc600fc5ecb'), 'company_name': 'Evergreen Credit Union', 'localisation': 'US'}
{'_id': ObjectId('66718e3343c23fc600fc5ecc'), 'company_name': 'Evergreen Credit Union', 'localisation': 'US'}
{'_id': ObjectId('66718e3343c23fc600fc5ecd'), 'company_name': 'Evergreen Credit Union', 'localisation': 'US'}
{'_id': ObjectId('66718e3343c23fc600fc5ece'), 'company_name': 'Evergreen Credit Union', 'localisation': 'US'}
{'_id': ObjectId('66718e3343c23fc600fc5ecf'), 'company_name': 'Evergreen Credit Union', 'localisation': 'US'}
{'_id': ObjectId('66718e3343c23fc600fc5ed0'), 'company_name': 'Eve

INDEXATION DE LA DONNEE VIA MONGODB

CREATION DE L'INDEX

In [1]:
from elasticsearch import Elasticsearch
from pymongo import MongoClient

# Connexion à MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['trustscore']  
collection = db['commentaires']  

# Connexion à Elasticsearch
es = Elasticsearch(['http://localhost:9200'])  # S'assurer que Elasticsearch est en cours d'exécution

# Fonction pour indexer les données dans Elasticsearch
def index_data():
    cursor = collection.find({}, {'_id': 0})  # Exclure le champ _id
    for document in cursor:
        # Insertion
        es.index(index='commentaires_index', body=document)

    print("Indexation terminée.")

# Appel de la fonction d'indexation
index_data()

# Fermer la connexion à MongoDB
client.close()


C:\Users\MSI KATANA B13V\AppData\Local\Temp\ipykernel_9972\1650875992.py:17: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  es.index(index='commentaires_index', body=document)


Indexation terminée.
